# Модель с использованием библиотеки `keras`

In [103]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import regularizers

model = Sequential()  # нейросеть последовательной архитектуры
model.add(Dense(3600, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape = (3,)))  # регуляризатор применят штраф: loss = l2 * reduce_sum(square(x))
model.add(Dense(1))

# Оптимизация Adam — это метод стохастического градиентного спуска, основанный на адаптивной оценке моментов первого и второго порядка.
# Согласно Kingma et al., 2014 , этот метод «эффективен в вычислительном отношении, требует мало памяти, инвариантен к диагональному масштабированию градиентов и хорошо подходит для задач, которые являются большими с точки зрения данных/параметров».
model.compile(optimizer=Adam(),loss='mse')  # loss = square(y_true - y_pred) == Средняя квадратическая ошибка

In [104]:
# просмотр построенной модели
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 3600)              14400     
                                                                 
 dense_5 (Dense)             (None, 1)                 3601      
                                                                 
Total params: 18,001
Trainable params: 18,001
Non-trainable params: 0
_________________________________________________________________
None


In [110]:
from functionXYZ import f_xy
import numpy as np


def prepare_random_data(degrees, elems_in_degree):
    """ Функция, генерирующая одинаковое число случайных элементов для каждой переданной степени.
    Обоснование. Если брать просто рандомные числа в пределах от [-1000; 1000],
    то чисел в пределах [-1;1] почти не будет, чисел в пределах [-10;10] будет очень мало
    и так далее по аналогии, что приведёт к необучению младших разрядов.
    Эту проблему и решает данная функция, возвращающая данные для тренировки модели с одинаковым числом нужных степеней.
    """
    
    for iters, degree in enumerate(degrees):
        degree_multyplier = pow(10, degree)
        X_data_tmp = np.random.rand(elems_in_degree, 2)*degree_multyplier*2-degree_multyplier
        X_data_tmp = np.append(X_data_tmp, np.ones((elems_in_degree, 1)), axis=1)
        X_data_tmp = X_data_tmp.reshape((elems_in_degree, 3))
        X_data = np.append(X_data, X_data_tmp, 0) if iters > 0 else X_data_tmp
    np.random.shuffle(X_data)
    y_data = np.array([
        f_xy(x[0], x[1]) for x in X_data
    ]).reshape((elems_in_degree*len(degrees), 1))
    return X_data, y_data

# это номера разрядов 10, в рамках которых будет обучаться модель (не совсем так, но приблежено к истине)
# не больше 4 разряда, на большее модель не хватает
degrees = [-1, 0, 0.5]  # на самом деле, модель нормально обучается не больше чем только на трёх последовательных разрядах, например 1, 2, 3
elems_in_degree = 5000  # число элементов на разряд
X_k, y_k = prepare_random_data(degrees, elems_in_degree)

In [112]:
# Тренировка модели, 20% идут на проверку
# Выполнить плитку 6-8 раз
hist = model.fit(
    X_k,
    y_k,
    validation_split=0.2,
    epochs=150,
    batch_size=512,
)

Epoch 1/150
24/24 [==============================] - 0s 6ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 2/150
24/24 [==============================] - 0s 4ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 3/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0017 - val_loss: 0.0015
Epoch 4/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 5/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0019 - val_loss: 0.0025
Epoch 6/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 7/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0015
Epoch 8/150
24/24 [==============================] - 0s 3ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 9/150
24/24 [==============================] - 0s 4ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 10/150
24/24 [==============================] - 0s 4ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 11/

In [ ]:
# сохранение модели
model.save('asl_model')

In [ ]:
# так очищали память в курсе от nvidia, здесь это выдаёт странную ошибку
import IPython


app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Использование обученной модели

In [ ]:
from tensorflow import keras


# загрузка модели
model = keras.models.load_model('asl_model')

In [114]:
from functionXYZ import f_xy
import numpy as np


def check_model(X):
    model_result = model.predict(X)
    function_result = [[f_xy(x[0], x[1])] for x in X]
    for i in range(len(X)):
        if function_result[i][0]:
            max_res = max(model_result[i][0], function_result[i][0])
            min_res = min(model_result[i][0], function_result[i][0])
            error = (max_res/min_res - 1)*100
        else:  # function_result[i][0] == 0
            error = np.absolute(model_result[i][0])*100
        print(
            f'Got: {model_result[i][0]}, want: {function_result[i][0]},'
            f' error: {error}%'
        )


# проверка нескольких примеров
# каждый пример = [x, y, 1]
check_X = [
    [23, 54, 1],
    [3, 7, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
    [0.32, 0.67, 1],
    [1, -2, 1],
    [1.01, -2.02, 1],
]

# этот пример также призван показать, что модель обучается только в конкретном диапозоне разрядов 10.
check_model(check_X)

1/1 [==============================] - 0s 20ms/step
Got: 608.4143676757812, want: 3620, error: 494.9892363372107%
Got: 68.45489501953125, want: 85, error: 24.16935264563356%
Got: 9.997167587280273, want: 10, error: 0.028332152032040092%
Got: 3.9990663528442383, want: 4, error: 0.023346628272302716%
Got: 9.001907348632812, want: 9, error: 0.021192762586808023%
Got: 7.5903496742248535, want: 7.5912999999999995, error: 0.012520184391151545%
Got: 0.0015122704207897186, want: 0, error: 0.15122704207897186%
Got: 0.0029009953141212463, want: 0.0005000000000000009, error: 480.1990628242483%


# Модель буз использования библиотек (legacy code)

Построенная модель без использования библиотек трудно настраивается на данные, не входящие в диапозон \[-1;1\].  
Призвана показать, как строится простейшая модель.

In [92]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(y):
    return y * (1.0 - y)


class NeuralNetwork:
    def __init__(self, params_number, h_number):
        self.weights1 = np.random.rand(params_number, h_number)
        self.weights2 = np.random.rand(h_number, 1)

    def feedforward(self, input_data):
        self.layer1 = sigmoid(np.dot(input_data, self.weights1))
        return sigmoid(np.dot(self.layer1, self.weights2))

    def backprop(self, input_data, output_data, check_data):
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        d_weights2 = np.dot(
            self.layer1.T,
            (2*(check_data - output_data) * sigmoid_derivative(output_data))
        )
        d_weights1 = np.dot(
            input_data.T,
            (np.dot(
                2*(check_data - output_data) * sigmoid_derivative(output_data),
                self.weights2.T
            ) * sigmoid_derivative(self.layer1))
        )
        # update the weights with the derivative (slope) of the loss function
        self.weights1 += d_weights1
        self.weights2 += d_weights2

    def train(self, epochs, input_data, check_data):
        output_data = np.zeros(input_data.shape[0])
        for _ in range(epochs):
            output_data = self.feedforward(input_data)
            self.backprop(input_data, output_data, check_data)

In [93]:
def func(x, y):
    return (x-1)**2+(y+2)**2

h_number = 6
params_number = 2
nn = NeuralNetwork(params_number, h_number)

In [94]:
epochs = 10000
values_number = 30
max_x = 10
# X = np.array([[0, 0], [0, 1], [1, 0], [1, 1], [-max_x, max_x]])
X = np.random.rand(values_number, 2)*max_x
y = np.array([
    func(x[0], x[1]) for x in X
]).reshape((values_number, 1))

max_el = max(np.amax(np.absolute(X)), np.amax(np.absolute(y)))
X = np.dot(X, 1/max_el)
y = np.dot(y, 1/max_el)

In [95]:
nn.train(epochs, X, y)

In [96]:
z = nn.feedforward(X)
z = np.dot(z, max_el)
y = np.dot(y, max_el)
for i in range(z.shape[0]):
    error = np.absolute(
        (np.absolute(z[i][0])-np.absolute(y[i][0]))
        / (np.absolute(z[i][0])+np.absolute(y[i][0]))
    )*100
    print(
        f'Got: {z[i][0]}, want: {y[i][0]},'
        f' error: {error}%'
    )

Got: 55.86547245682782, want: 85.32176994698325, error: 20.86328551265785%
Got: 61.49856361753632, want: 64.03659068299694, error: 2.021765998219547%
Got: 7.170559847577435, want: 9.17737650827642, error: 12.275657410302964%
Got: 139.6569293971853, want: 140.72095592618874, error: 0.3794973087040129%
Got: 150.23073050446288, want: 164.98484836654234, error: 4.680643613784469%
Got: 143.79623999717776, want: 147.68750975075648, error: 1.334986858424786%
Got: 16.105155264218016, want: 18.156545505300286, error: 5.987415087424195%
Got: 146.4976684778596, want: 158.56977527088105, error: 3.9571927586492204%
Got: 160.1252015512418, want: 187.3637241262987, error: 7.838673569797004%
Got: 132.4643700132971, want: 135.6398672783728, error: 1.184426362355881%
Got: 141.0518672522793, want: 142.52215822368098, error: 0.518485770667428%
Got: 23.776866853634758, want: 28.735287531989567, error: 9.442424780256628%
Got: 62.721584786086446, want: 80.42077550861036, error: 12.3647470155483%
Got: 132.846